<h4> Feature selection (특징 선택) <h4>
<img src="fig_source/feature_selection_1.png"></img>
    
- 모델을 구성하는 주요 피처들을 선택하는 feature engineering 과정.
    - 데이터 중 Target 예측 과정에서 중요한 정보와 중요하지 않은 정보를 분류하여 모델 학습에 적합한 데이터로 선별하는 과정을 의미함.
    - 불필요한 다수의 피처들로 인해 모델 성능을 떨어뜨릴 가능성 제거
    - 설명 가능한 모델이 될 수 있도록 피처들을 선별
    - 다수의 피처로 모델 학습 시 발생하는 과적합 이슈 방지

    - *feature engineering: 원본 데이터를 변형하거나 새로운 특성을 생성하는 과정으로, 주로 데이터를 보다 유용한 형태로 만드는 데 중점
        - 변환: 로그 변환, 정규화, 표준화 등 데이터 변환.
        - 조합: 여러 변수를 결합하여 새로운 변수를 생성 (예: 비율, 차이 등).
        - 특성 생성: 도메인 지식에 기반한 새로운 특성 추가.
        - 결측치 처리: 결측 데이터를 처리하여 데이터 품질 향상.


<img src="fig_source/feature_selection_2.png"></img>


In [ ]:
import os
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

In [ ]:
# Loading data & checking sample
df = pd.read_csv(os.path.join('data',"Radiomic_features_all.csv") , sep = ",")
print(f'Data shape: {df.shape}')
print(f'Features: {df.columns.to_list()}')
display(df)

In [ ]:
# Data 
X = df.drop('label', axis=1) # Features
Y = df['label']  # Target variable (label)

<img src="fig_source/feature_selection_filter.png"></img>
- 개요: Filter 방법은 데이터의 통계적 속성에 기반하여 특징을 평가하고 선택하는 방식. 다른 변수와의 상관관계 없이 각 특징 자체만의 특성을 고려하므로 빠르게 계산 가능함.
- 특징: 
    - 독립적인 변수 분석에 기반해 특징을 필터링하므로, 계산 비용이 낮아 빠르게 실행가능.
    - 모델에 독립적이기 때문에 데이터에 일반화할 수 있지만, 복잡한 특징 간 상호작용을 반영하기 어려움.

- ex) 
    - Chi-square Test: 범주형 데이터에서 feature와 타겟 간 독립성 테스트. p-값을 통해 각 특징의 유의성을 평가하며, 낮은 p-값을 가지는 변수는 타겟과의 관계가 유의미하다고 판단
    - Variance Thresholding: 분산이 매우 낮은 특징은 정보가 거의 없다고 판단하여 제거
    - Correlation Coefficient: 피어슨 상관계수로 feature와 타겟 간 관계를 평가

In [ ]:
# 1. Filter Methods: Variance Thresholding
"""
대표 예시 - Variance Thresholding: 분산이 매우 낮은 특징은 정보가 거의 없다고 판단하여 제거.
"""

from sklearn.feature_selection import VarianceThreshold

# 분산이 0.1 미만인 특징을 제거.
selector = VarianceThreshold(threshold=0.1)
X_selected = selector.fit_transform(X)  # X는 특징 데이터셋
print("Variance Thresholding 후 남은 특징 수:", X_selected.shape[1])

var_selected_features = X.columns[selector.get_support()]
print(var_selected_features)

<img src="fig_source/feature_selection_wrapper.png"></img>
- 2. Wrapper Methods: Recursive Feature Elimination (RFE)

- 개요: Wrapper 방법은 특정 모델의 성능을 기준으로 특징 조합을 평가하고 선택함. 특징 조합을 테스트하면서 모델 성능이 가장 높은 특징 조합을 찾는 방법.
- 특징: 
    - 모델 학습을 반복하여 가장 좋은 조합을 찾기 때문에 성능이 좋지만, 계산 비용이 높음.
    - 특히 상호작용이 복잡한 데이터에 대해 효과적이며, 특정 모델에 최적화된 조합의 subset 특징을 선별 가능함.

- ex) 
    - Forward Selection: 빈 feature 세트에서 하나씩 추가하며 성능 평가.
    - Backward Elimination: 모든 feature로 시작해 기여도 낮은 feature를 하나씩 제거.
    - Recursive Feature Elimination (RFE): 모델이 덜 중요한 특징을 반복적으로 제거하여 최적의 특징을 선별함.

In [ ]:
# 2. Wrapper Methods: Recursive Feature Elimination (RFE)
"""

대표 예시 - Recursive Feature Elimination (RFE): 모델이 덜 중요한 특징을 반복적으로 제거하여 최적의 특징을 선별함. 주로 회귀나 분류 모델과 결합하여 사용함.
"""
from sklearn.feature_selection import RFE
from sklearn.ensemble import RandomForestClassifier

# RandomForest와 결합하여 상위 10개 특징 선택
rf_model = RandomForestClassifier()
selector = RFE(rf_model, n_features_to_select=21)
X_selected = selector.fit_transform(X, Y)
print("RFE 후 남은 특징 수:", X_selected.shape[1])
var_selected_features = X.columns[selector.get_support()]
print(var_selected_features)

<img src="fig_source/feature_selection_embedded.png"></img>
- 3. Embedded Methods: Tree-based Feature Importance (Random Forest)

- 개요: Embedded 방법은 모델 학습 과정에서 중요한 특징을 동시에 선택하는 방법. 모델이 학습을 진행하면서 특징의 중요도를 평가하고, 특정 기준 이상인 특징만 선별.
- 특징:
    - 학습과 선택이 동시에 이루어지기 때문에 계산 비용이 적고, 모델 성능에 큰 영향을 줄 수 있는 특징을 자동으로 선택함.
    - 모델에 내장된 특징 선택 기능을 활용하므로, 특정 모델에 최적화되어 일반화가 어려울 수 있음.

- ex) 
    - Lasso (Least Absolute Shrinkage and Selection Operator): L1 규제를 적용하여 중요하지 않은 변수의 계수를 0으로 만들어 특징 선택을 수행
    - Random Forest Feature Importance: Random Forest와 같은 Tree-based 모델은 특징의 중요도(feature importances) 추출이 가능함.

- 랜덤 포레스트(Random Forest): 다수의 결정 트리(Decision Trees)를 조합하여 예측 성능을 높이는 앙상블 학습(Ensemble Learning) 기법 중 하나
* 결정 트리는 데이터를 분할해가면서 각 노드에서 결정 규칙을 통해 최종 예측을 내리는 트리 구조의 모델
- 랜덤포레스트의 주요 특징
    - 부트스트랩핑(Bootstrap Sampling): 학습 데이터를 무작위로 샘플링하여 각 트리의 학습 데이터 세트를 구성합니다. 이렇게 샘플링된 데이터는 원래 데이터와 조금씩 다르기 때문에 각 트리가 서로 다른 관점을 학습

    - 특징 무작위성: 각 트리가 분기할 때 사용하는 특징(특성)도 무작위로 선택. 이는 트리들이 서로 상관관계를 줄여 다르게 학습되도록 하는 데 도움.

    - 다수결 투표: 분류 문제의 경우 각 트리의 예측 결과 중 가장 많이 나온 클래스를 최종 예측

In [ ]:
# 3. Embedded Methods: Tree-based Feature Importance (Random Forest)
"""
개요: Embedded 방법은 모델 학습 과정에서 중요한 특징을 동시에 선택하는 방법. 모델이 학습을 진행하면서 특징의 중요도를 평가하고, 특정 기준 이상인 특징만 선별.
특징:
학습과 선택이 동시에 이루어지기 때문에 계산 비용이 적고, 모델 성능에 큰 영향을 줄 수 있는 특징을 자동으로 선택함.
모델에 내장된 특징 선택 기능을 활용하므로, 특정 모델에 최적화되어 일반화가 어려울 수 있음.
대표 예시 - Random Forest Feature Importance: Random Forest와 같은 Tree-based 모델은 특징의 중요도(feature importances) 추출이 가능함. 이 중요도 정보를 기반으로 지정된 수만큼의 상위 특징만 선택함.
"""


from sklearn.ensemble import RandomForestClassifier
import numpy as np

rf_model = RandomForestClassifier()
rf_model.fit(X, Y)
importances = rf_model.feature_importances_
indices = np.argsort(importances)[-10:][::-1]  # 중요도를 기준으로 내림차순 정렬하여 상위 10개 인덱스 선택
X_selected = X.iloc[:,indices]
print("Random Forest Feature Importance 기반 선택 후 남은 특징 수:", X_selected.shape[1])
var_selected_features = X.columns[indices]
print(var_selected_features)
selected_importances = importances[indices]
print(selected_importances.shape)
print(var_selected_features.shape)

In [ ]:
# 상위 10개 특징 시각화
plt.figure(figsize=(10, 6))
plt.barh(var_selected_features, selected_importances, color='skyblue')
plt.xlabel("Importance")
plt.title("Top 10 Features based on Tree-based Feature Importance")
plt.gca().invert_yaxis()  # y축을 뒤집어 가장 중요한 특징이 위로 오게 설정
plt.tight_layout()
plt.savefig("top_10_tree_based_features.png")
plt.show()